### Test notebook for development

In [1]:
try:
    import os
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/School/DS-GA 1011/capstone')
except:
    pass

Mounted at /content/drive


In [10]:
# ! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
try:
  from google.colab import userdata
  hf_token = userdata.get_secret('HUGGING_FACE_TOKEN')
except:
  import os
  hf_token = os.getenv('HUGGING_FACE_TOKEN')
hf_login_command = f'huggingface-cli login --token {hf_token} --add-to-git-credential'
os.system(hf_login_command)

256

**Testing the model**

In [4]:
from src.model import AutoencoderConfig, SparseAutoencoder
from src.trainer import TrainingConfig, MonosemanticityTrainer
import torch

In [5]:
model_config = AutoencoderConfig(
     input_dim=768,
    hidden_dim=1024,
)
trainer_config = TrainingConfig(
    batch_size=64,
    num_epochs= 10,
    mixed_precision= 'fp16',
    run_path='tests/',
    learning_rate=1e-4
)

In [6]:
# model and optimizer
model = SparseAutoencoder(model_config)
optimizer = torch.optim.Adam(model.parameters(), lr=trainer_config.learning_rate)
trainer = MonosemanticityTrainer(model, optimizer=optimizer, train_config=trainer_config)

In [7]:
# sample data
normal_dist = torch.distributions.Normal(loc=0, scale=1)
samples = normal_dist.sample(sample_shape=torch.Size([512,model_config.input_dim]))
dataset = torch.utils.data.TensorDataset(samples)
train_loader = torch.utils.data.DataLoader(dataset, batch_size = trainer_config.batch_size, shuffle=True)

In [8]:
# training
trainer.train(train_loader)

INFO:src.trainer:Saving model to: /content/drive/MyDrive/School/DS-GA 1011/capstone/tests/model.pkl
  0%|          | 0/10 [00:00<?, ?it/s]INFO:src.trainer:
Epoch 1/10
DEBUG:src.trainer:Training epoch
DEBUG:src.evaluation:MSE Loss: tensor([1.0703, 1.1187, 1.0117, 1.1059, 1.0727], grad_fn=<SliceBackward0>)
DEBUG:src.evaluation:L1 Regularization: tensor([0.0251, 0.0262, 0.0249, 0.0227, 0.0239], grad_fn=<SliceBackward0>)
DEBUG:src.evaluation:Total Loss: tensor([1.0954, 1.1448, 1.0366, 1.1287, 1.0966], grad_fn=<SliceBackward0>)
DEBUG:src.evaluation:MSE Loss: tensor([1.0883, 1.0583, 1.0152, 1.1222, 1.1255], grad_fn=<SliceBackward0>)
DEBUG:src.evaluation:L1 Regularization: tensor([0.0249, 0.0243, 0.0227, 0.0258, 0.0233], grad_fn=<SliceBackward0>)
DEBUG:src.evaluation:Total Loss: tensor([1.1132, 1.0826, 1.0378, 1.1480, 1.1488], grad_fn=<SliceBackward0>)
DEBUG:src.evaluation:MSE Loss: tensor([0.9050, 1.0749, 1.0581, 1.0758, 1.0478], grad_fn=<SliceBackward0>)
DEBUG:src.evaluation:L1 Regularizati

{'train_loss': [1.0712919235229492,
  1.037842869758606,
  1.016418695449829,
  0.9976164102554321,
  0.9795904755592346,
  0.9619669318199158,
  0.9449439644813538,
  0.9284594058990479,
  0.9123886227607727,
  0.8965653777122498],
 'train_mse_loss': [1.0477808713912964,
  1.014610767364502,
  0.9933608770370483,
  0.9746859669685364,
  0.9567487835884094,
  0.9391788840293884,
  0.922172486782074,
  0.9056698679924011,
  0.8895455598831177,
  0.8736357092857361],
 'train_l1_regularization': [0.02351101115345955,
  0.023232044652104378,
  0.023057783022522926,
  0.022930428385734558,
  0.022841691970825195,
  0.02278801240026951,
  0.022771436721086502,
  0.022789573296904564,
  0.02284299023449421,
  0.022929610684514046]}

**Testing data generation**

In [11]:
from src.dataset import ActivationDataset, DataConfig, GPT2ActivationExtractor, GemmaActivationExtractor

dt_config = DataConfig(
    model_name='gpt2',
    use_flash_attention=False
)
extractor = GPT2ActivationExtractor(dt_config)


INFO:src.dataset:Using gpt2 extractor on cpu


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
gpt2_extractor = GPT2ActivationExtractor(dt_config)


In [ ]:
res = gpt2_extractor.extract_activations(['I love you'], 5)
res['activations'], res['activations'].shape

In [ ]:
import torch.multiprocessing as mp

try:
    mp.set_start_method('spawn')
except RuntimeError:
    pass # method has already been set


dataset = ActivationDataset(
    extractor=gpt2_extractor,
    dataset_name="RealTimeData/wikitext_latest",
    split="train",
    text_column="text"
)
dataloader = dataset.get_dataloader(batch_size=2)

In [ ]:
batch = next(iter(dataloader))
batch